[View in Colaboratory](https://colab.research.google.com/github/UjjwalSaxena/Simple-eye-detection-tests-ML/blob/master/main.ipynb)

In [89]:
# !wget https://github.com/UjjwalSaxena/Simple-eye-detection-tests-ML/blob/master/dataset_B_Eye_Images.zip 
# ! git clone https://github.com/UjjwalSaxena/Simple-eye-detection-tests-ML
! wget https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Image_created_with_a_mobile_phone.png/1200px-Image_created_with_a_mobile_phone.png

--2018-08-03 06:10:48--  https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Image_created_with_a_mobile_phone.png/1200px-Image_created_with_a_mobile_phone.png
Resolving upload.wikimedia.org (upload.wikimedia.org)... 198.35.26.112, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|198.35.26.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1594447 (1.5M) [image/png]
Saving to: ‘1200px-Image_created_with_a_mobile_phone.png’

1200px-Image_create 100%[===================>]   1.52M  3.14MB/s    in 0.5s    

2018-08-03 06:10:49 (3.14 MB/s) - ‘1200px-Image_created_with_a_mobile_phone.png’ saved [1594447/1594447]



In [0]:
# !pip install patool
# !pip install pyunpack
# !mkdir "EyeImages"
# !ls -lrt Simple-eye-detection-tests-ML/dataset_B_Eye_Images.zip

# !rm "dataset_B_Eye_Images.zip.1"

In [0]:
# from pyunpack import Archive
# Archive('dataset_B_Eye_Images.zip').extractall('EyeImages')
# import patoolib
# patoolib.extract_archive("dataset_B_Eye_Images.rar", outdir="EyeImages")
# !pip install unrar
# from unrar import rarfile
# rar = rarfile.RarFile('dataset_B_Eye_Images.rar')
# rar.namelist()

In [0]:

def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())

upload_files()

In [53]:
# Archive('dataset_B_Eye_Images.zip').extractall('EyeImages')
# import zipfile
# zip_ref = zipfile.ZipFile('Simple-eye-detection-tests-ML/dataset_B_Eye_Images.zip', 'r')
# zip_ref.extractall('EyeImages')
# zip_ref.close()
# !ls -lrt 
!ls -lrt "EyeImages/dataset_B_Eye_Images"

total 264
drwxr-xr-x 2 root root 69632 Aug  3 05:30 closedLeftEyes
drwxr-xr-x 2 root root 69632 Aug  3 05:30 closedRightEyes
drwxr-xr-x 2 root root 65536 Aug  3 05:30 openLeftEyes
drwxr-xr-x 2 root root 65536 Aug  3 05:30 openRightEyes


In [0]:
import matplotlib.pyplot as plt
import cv2 
import glob
import numpy as np

In [0]:

def loadImages(path):
  return plt.imread(path)

def getImagesFromFolder(foldername):
  imagePaths = [image_path for image_path in glob.glob(foldername+'/*.jpg')]
#   images= [loadImages(image_path) for image_path in imagePaths]
  return imagePaths

In [0]:
# cle= getImagesFromFolder("EyeImages/dataset_B_Eye_Images/closedLeftEyes")
# cre= getImagesFromFolder("EyeImages/dataset_B_Eye_Images/closedRightEyes")
# ole= getImagesFromFolder("EyeImages/dataset_B_Eye_Images/openLeftEyes")
# ore= getImagesFromFolder("EyeImages/dataset_B_Eye_Images/openRightEyes")



In [0]:
from skimage.feature import hog
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    # Call with two outputs if vis==True
  if vis == True:
    features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                              cells_per_block=(cell_per_block, cell_per_block),transform_sqrt=False, 
                              visualise=vis, feature_vector=feature_vec)
    return features, hog_image
  # Otherwise call with one output
  else:     
    
    features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                   cells_per_block=(cell_per_block, cell_per_block),transform_sqrt=False, 
                   visualise=vis, feature_vector=feature_vec)
    
    return features
      
      
def get_histogram_features(img):
  c1= np.histogram(img, bins=8, range=(0,256))
  return c1



def get_image_features(img, color_space='RGB', size=(24, 24)):      
  features = cv2.resize(img, size).ravel() 
  return features
  
def get_features(img):
  x = np.array(get_hog_features(img, 7, 6, 1))
  y = get_image_features(img)
  return np.concatenate((x,y))

In [177]:
features_open=[]
features_closed=[]

eye_closed= cle+ cre
eye_open= ole+ ore
# print(len(eye_closed))


for eye_path in eye_closed:
  eye= loadImages(eye_path.strip())
  features_closed.append(get_features(eye))
  
for eye_path in eye_open:
  eye= loadImages(eye_path.strip())
  features_open.append(get_features(eye))


/usr/local/lib/python3.6/dist-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [0]:
X = np.vstack((features_open, features_closed)).astype(np.float64)
y = np.hstack((np.ones(len(features_open)), np.zeros(len(features_closed))))



In [179]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rand_state)
X_scaler = StandardScaler().fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)


print('Feature vector length:', len(X_train[0]))

Feature vector length: 688


In [180]:
%%time
from sklearn.svm import SVC

print('training started') 
clf = SVC()
clf.fit(X_train, y_train)
print('Test Accuracy of SVC = ', round(clf.score(X_test, y_test), 4))

training started
Test Accuracy of SVC =  0.9557
CPU times: user 7.55 s, sys: 4.85 ms, total: 7.55 s
Wall time: 7.55 s


In [0]:
def predict(img_portion):
  features= get_features(img_portion)
  features= X_scaler.transform(np.array(features).reshape(1, -1))
  prediction= clf.predict(features)
  return prediction

In [211]:
All_Windows=[]

def generate_all_windows(img, size=24, overlap=0.3):
  height= img.shape[0]
  width= img.shape[1]
  
  for i in range(size,height-size,int(size*(1-overlap))):
    for j in range(size,width-size,int(size*(1-overlap))):
      location=(i,j)
      All_Windows.append((i,i+size, j, j+size, location))
      
generate_all_windows(loadImages("1200px-Image_created_with_a_mobile_phone.png"),24, 0.2)
# print(len(All_Windows))

def generate_windows_around_prev(img, location, size=24, overlap=0.3, area=50):
  height= img.shape[0]
  width= img.shape[1]
  current_windows=[]
  for i in range(location[0]-area,location[0]+area,int(size*(1-overlap))):
    for j in range(location[1]-area,location[1]+area,int(size*(1-overlap))):
      location=(i,j)
      current_windows.append((i,i+size, j, j+size, location))
  return current_windows


centroid=()
distracted=0
def Detect(img):
  img= cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  size=24

  if(centroid==())
    generate_all_windows(img,24, 0.2)
    for window in All_Windows:
      location= window[4]
      p= predict(img[location[0]:location[0]+size, location[1]:location[1]+size])
      if p==1:
        distracted= max(distracted-1,0)
        centroid=location
        break
      elif p==0:
        distracted+=1
        centroid=location
        break
      if distracted>5:
        print("distracted")
  else:
    anchor=False
    current_windows= generate_windows_around_prev(img, centroid, overlap=0.2)
    for window in current_windows:
      location= window[4]
      p= predict(img[location[0]:location[0]+size, location[1]:location[1]+size])
      if p==1:
        distracted= max(distracted-1,0)
        centroid=location
        break
      elif p==0:
        distracted+=1
        centroid=location
        break
      if distracted>5:
        print("distracted")
    if !anchor:
      centroid=()
      
  

25


/usr/local/lib/python3.6/dist-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


5.579438209533691
